In [2]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/


def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df

#benchmark risk

def benchmark(indices):
    indices = indices
    start = date.today() - timedelta(days = 3650*2)
    end = date.today()

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

In [162]:
tickers = ['SPY', 'IWM', 'QQQ', 'GDX', 'GLD', 'USO','TLT', 'LQD']
start = date.today() - timedelta(days = 3650*2)
end = date.today()
df = pd.DataFrame([web.DataReader(ticker, 'yahoo', start, end)['Adj Close'] for ticker in tickers]).T
df.columns = tickers

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

benchmarks = benchmark(['SPY', 'IWM', 'QQQ', 'VT'])

In [163]:
factor = 1.5
max_risk = factor*benchmarks[benchmarks['benchmark']=='SPY']['stdev']

results_frame[results_frame['stdev']<float(max_risk)]

,ret,stdev,sharpe,SPY,IWM,QQQ,GDX,GLD,USO,TLT,LQD
0,0.068531,0.103303,0.663397,0.069154,0.159038,0.077283,0.015648,0.221926,0.097974,0.311932,0.047044
1,0.072255,0.136695,0.528584,0.055677,0.321654,0.020750,0.124469,0.083733,0.086788,0.278777,0.028152
2,0.034486,0.225137,0.153176,0.023298,0.008137,0.263553,0.272420,0.140167,0.272363,0.019637,0.000425
3,0.071212,0.144528,0.492717,0.005979,0.045658,0.110161,0.224477,0.112117,0.074506,0.196021,0.231081
4,0.078324,0.164975,0.474762,0.128587,0.115129,0.172065,0.186276,0.171851,0.052272,0.020911,0.152910
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.065064,0.117438,0.554026,0.098889,0.139287,0.095560,0.053917,0.180277,0.100337,0.155771,0.175961
9996,0.055880,0.129451,0.431667,0.033027,0.132483,0.133442,0.093698,0.114175,0.141295,0.154652,0.197228
9997,0.033042,0.158991,0.207825,0.022661,0.235775,0.004238,0.104879,0.103185,0.249195,0.109752,0.170316
9998,0.074671,0.175490,0.425502,0.004760,0.072271,0.256396,0.257710,0.097816,0.078254,0.131025,0.101769


In [138]:
results_frame['levered_ret']=[0.0]*len(results_frame)
results_frame['levered_stdev']=[0.0]*len(results_frame)
results_frame['levered_sharpe'] = [0.0]*len(results_frame)

In [159]:
leverage = 1.5
margin_int =0.015
for i in range(len(results_frame)):
    results_frame['levered_ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
    results_frame['levered_stdev'][i] = results_frame['stdev'][i] * leverage
    results_frame['levered_sharpe'][i] = results_frame['levered_ret'][i]/results_frame['levered_stdev'][i]

max_levered_sharpe = results_frame['levered_sharpe'].max()
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['levered_sharpe']==max_levered_sharpe]

,ret,stdev,sharpe,SPY,IWM,QQQ,GDX,GLD,USO,TLT,LQD,levered_ret,levered_stdev,levered_sharpe
4657,0.101982,0.073877,1.380421,0.124211,0.080898,0.121102,0.000698,0.07878,0.000932,0.323587,0.269792,0.145473,0.110816,1.312741


In [156]:
benchmarks[benchmarks['benchmark']=='SPY']

,benchmark,ret,stdev,sharpe
1,SPY,0.143769,0.17096,0.840954
